In [30]:
!python --version

Python 3.10.12


In [31]:
!pip install tensorflow
print("done")

done


In [32]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.17.1


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import pandas as pd

# Read the Excel file into a DataFrame
df = pd.read_csv("drive/MyDrive/data.csv")

# Print the first 5 rows of the DataFrame
print(df.head())

                                                 URL  \
0                                 http://rgipt.ac.in   
1                                http://www.mutuo.it   
2         http://vamoaestudiarmedicina.blogspot.com/   
3  https://www.astrologyonline.eu/Astro_MemoNew/P...   
4        https://www.lifewire.com/tcp-port-21-818146   

                                             Content      status  
0  \n\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1...  legitimate  
1  \n\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="it"...  legitimate  
2  <!DOCTYPE html>\n<html class='v2' dir='ltr' la...  legitimate  
3  <meta name='viewport' content='width=device-wi...  legitimate  
4  <!DOCTYPE html>\n<html id="structuredContentTe...  legitimate  


In [35]:
length_url = len(df.URL)
print(length_url)

5489


In [36]:
copy_url = []
copy_html = []
copy_status = []
legitimate_counter = 0
phishing_counter = 0
total_count = 0

for i in range(length_url):
    copy_url.append(df.URL[i])
    copy_html.append(df.Content[i])
    copy_status.append(df.status[i])
    if copy_status[i] == "legitimate":
        legitimate_counter += 1
    else:
        phishing_counter += 1

print(legitimate_counter)
print(phishing_counter)
print(legitimate_counter + phishing_counter)

4394
1095
5489


In [37]:
binary_html = []
error_index = []

# Loop through URLs
i = 0  # Start from the first index
while i < length_url:
    try:
        # Convert HTML content to binary
        res = ''.join(format(b, '08b') for b in bytearray(copy_html[i], encoding='utf-8'))
        binary_html.append(res)
        print(f"Finished {i + 1} / {length_url}")
        i += 1  # Move to the next index only if successful
    except (TypeError):
        # Log error index
        print(f"Error at {i + 1}")
        error_index.append(i + 1)

        # Remove erroneous entries from all lists
        del copy_html[i]
        del copy_url[i]
        del copy_status[i]

        # Decrement length_url to account for the removed item
        length_url -= 1

# Verify the lists are updated
print(f"Updated lengths: binary_html={len(binary_html)}, copy_html={len(copy_html)}, copy_url={len(copy_url)}, copy_status={len(copy_status)}")

Streaming output truncated to the last 5000 lines.
Finished 490 / 5488
Finished 491 / 5488
Finished 492 / 5488
Finished 493 / 5488
Finished 494 / 5488
Finished 495 / 5488
Finished 496 / 5488
Finished 497 / 5488
Finished 498 / 5488
Finished 499 / 5488
Finished 500 / 5488
Finished 501 / 5488
Finished 502 / 5488
Finished 503 / 5488
Finished 504 / 5488
Finished 505 / 5488
Finished 506 / 5488
Finished 507 / 5488
Finished 508 / 5488
Finished 509 / 5488
Finished 510 / 5488
Finished 511 / 5488
Finished 512 / 5488
Finished 513 / 5488
Finished 514 / 5488
Finished 515 / 5488
Finished 516 / 5488
Finished 517 / 5488
Finished 518 / 5488
Finished 519 / 5488
Finished 520 / 5488
Finished 521 / 5488
Finished 522 / 5488
Finished 523 / 5488
Finished 524 / 5488
Finished 525 / 5488
Finished 526 / 5488
Finished 527 / 5488
Finished 528 / 5488
Finished 529 / 5488
Finished 530 / 5488
Finished 531 / 5488
Finished 532 / 5488
Finished 533 / 5488
Finished 534 / 5488
Finished 535 / 5488
Finished 536 / 5488
Finished 

In [38]:
from PIL import Image
import numpy as np

binary_images = []  # This will store the final images after conversion

def binary_to_grayscale_rgb_image(binary_string, width, height):
    """
    Converts a binary string to a grayscale image with 3 channels (RGB),
    padding the string with zeros if it's too short and handling invalid characters.

    Args:
        binary_string (str): Input binary string.
        width (int): Width of the image.
        height (int): Height of the image.

    Returns:
        PIL.Image: Grayscale image in RGB format.
    """
    # Calculate the required length of the binary string
    required_length = width * height * 8  # Each pixel needs 8 bits (grayscale)

    # Pad the binary string with zeros if it's too short
    binary_string = binary_string.ljust(required_length, '0')

    # Create a numpy array to hold the RGB image data
    img_data = np.zeros((height, width, 3), dtype=np.uint8)

    # Fill the image data
    idx = 0
    for row in range(height):
        for col in range(width):
            # Extract 8 bits for the grayscale value
            gray = 0  # Default to 0 for invalid data
            if idx + 8 <= len(binary_string) and all(c in '01' for c in binary_string[idx:idx + 8]):
                gray = int(binary_string[idx:idx + 8], 2)
            # Set the grayscale value across all RGB channels
            img_data[row, col] = [gray, gray, gray]
            idx += 8  # Move to the next 8 bits

    # Convert the numpy array to a PIL image
    img = Image.fromarray(img_data, mode='RGB')  # 'RGB' mode for 3-channel images
    return img

# Assuming `binary_html` is a list of your binary data strings
# The list of binary strings needs to be defined in your program
for i in range(len(binary_html)):  # Iterate over your binary data list
    binary_string = binary_html[i]
    width = 28  # Fixed width for 28x28 image
    height = 28  # Fixed height for 28x28 image

    try:
        # Convert binary string to grayscale RGB image
        img = binary_to_grayscale_rgb_image(binary_string, width, height)

        # Resize the image (if needed, but this is optional as it already fits 28x28)
        resized_image = img.resize((28, 28))

        # Append the resulting image to the list
        binary_images.append(resized_image)

        print(f"Finished processing image {i + 1}")

    except Exception as e:
        print(f"Error processing image {i + 1}: {e}")

# Optional: Display the first image to verify
if binary_images:
    binary_images[0].show()


Streaming output truncated to the last 5000 lines.
Finished processing image 463
Finished processing image 464
Finished processing image 465
Finished processing image 466
Finished processing image 467
Finished processing image 468
Finished processing image 469
Finished processing image 470
Finished processing image 471
Finished processing image 472
Finished processing image 473
Finished processing image 474
Finished processing image 475
Finished processing image 476
Finished processing image 477
Finished processing image 478
Finished processing image 479
Finished processing image 480
Finished processing image 481
Finished processing image 482
Finished processing image 483
Finished processing image 484
Finished processing image 485
Finished processing image 486
Finished processing image 487
Finished processing image 488
Finished processing image 489
Finished processing image 490
Finished processing image 491
Finished processing image 492
Finished processing image 493
Finished processing

In [44]:
#phishing = 0; legitimate = 1
status_as_binary = []
converted_legitimate = 0
converted_phishing = 0

for i in range(len(copy_status)):
    if copy_status[i] == "legitimate":
        status_as_binary.append(1)
        converted_legitimate += 1
    else:
        status_as_binary.append(0)
        converted_phishing += 1

print(legitimate_counter)
print(converted_legitimate)

print(phishing_counter)
print(converted_phishing)

print(len(status_as_binary))
print(len(copy_status))

4394
4382
1095
1080
5462
5462


In [40]:
copy_images = []
copy_status_binary = []

for i in range(len(binary_images)):
    copy_images.append(binary_images[i])

for i in range(len(status_as_binary)):
    copy_status_binary.append(status_as_binary[i])

print("done")
print(len(copy_status_binary))

done
5462


In [41]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split

images = np.array(copy_images)
labels = np.array(copy_status_binary)

print("Shape of images:", images.shape)
print("Shape of labels:", labels.shape)

print("Done Creating Arrays")

dataset = tf.data.Dataset.from_tensor_slices((images, labels))

print("Done Creating Dataset")

for img, label in dataset:
    print(f"Image shape: {img.shape}, Label: {label}")

train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42)

# Check the shapes of the split datasets
print(f"Train Images Shape: {train_images.shape}")
print(f"Test Images Shape: {test_images.shape}")
print(f"Train Labels Shape: {train_labels.shape}")
print(f"Test Labels Shape: {test_labels.shape}")

Streaming output truncated to the last 5000 lines.
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 0
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 0
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 0
Image shape: (28, 28, 3), Label: 0
Image shape: (28, 28, 3), Label: 0
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 0
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 0
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Image shape: (28, 28, 3), Label: 1
Imag

In [42]:
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
testing_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

batch_size = 32
training_dataset = training_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Example Model (Simple CNN)
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(28, 28, 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the dataset
model.fit(training_dataset, epochs=100)
print("Done")

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


137/137 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - accuracy: 0.7648 - loss: 6.9926
Epoch 2/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.8715 - loss: 0.3599
Epoch 3/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.8794 - loss: 0.3400
Epoch 4/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.9101 - loss: 0.2633
Epoch 5/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.9140 - loss: 0.2467
Epoch 6/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.9240 - loss: 0.2135
Epoch 7/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.9263 - loss: 0.1969
Epoch 8/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9419 - loss: 0.1644
Epoch 9/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - accuracy: 0.9464 - loss: 0.1528
Epoch 10/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.9652 - loss: 0.1063
Epoch 11/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 36ms/step - accuracy: 0.9612 - loss: 0.1094
Epoch 12/100
137/137 ━━━━━━━━━━━━━━━━━━━━ 

In [43]:
test_images_resized = tf.image.resize(test_images, (28, 28))

# Ensure the data type is correct (float32)
test_images_resized = tf.cast(test_images_resized, tf.float32)

# Normalize if necessary
test_images_resized /= 255.0  # Normalize to the range [0, 1] if required

# Create the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((test_images_resized, test_labels))

# Batch the dataset
test_dataset = test_dataset.batch(32)

# Now evaluate the model
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7753 - loss: 0.6075
Test Loss: 0.6026083827018738
Test Accuracy: 0.7904849052429199
